---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [5]:
import pandas as pd
import numpy as np
import re

def blight_model():
    
    # Your code here
    # Importing the data
    df = pd.read_csv('train.csv', encoding='ISO-8859-1')
    df1 = pd.read_csv('test.csv')
    # Selecting data from USA only
    USA_df = df.query('country == "USA"')
    # Creating a List of columns to drop , from columns with future dates to other data that points 
    # to future occurence is order to avoid data leakage
    cols_to_drop = ['violation_zip_code','mailing_address_str_number','mailing_address_str_name'
                    ,'state','non_us_str_code','hearing_date','violation_description','clean_up_cost',
                    'balance_due','payment_amount','payment_date','payment_status','collection_status',
                    'grafitti_status','country','compliance_detail','violator_name']
    # Dropping these columns 
    cols_df = USA_df.drop(cols_to_drop, axis=1)
    # Remove all rows with null values
    no_null = cols_df.dropna()    
    # The columns to keep in the test dataset
    col_to_keep = cols_df.columns[:-1]
    test = df1[col_to_keep] 
    # Make ticket_id the index in both train and test dataset
    no_null = no_null.set_index('ticket_id')
    test = test.set_index('ticket_id')
    
    # Split the preprocessed train.csv into feature and target tables
    train = no_null.iloc[:,:-1]
    Y = no_null.iloc[:,-1]
    
    # importing all the needed sklearn functions
    from sklearn.model_selection import train_test_split 
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.preprocessing import LabelEncoder
    
    # Applying the label encoding on the categorical features
    for x in list(train.columns):
        le = LabelEncoder()
        if train[x].astype(str).str.contains(r'[\-A-Z]+',flags=re.IGNORECASE, regex = True).any() == True :
            train[x] = le.fit_transform(np.array(train[x].values).astype(str))
            test[x] = le.fit_transform(np.array(test[x].values).astype(str))
   
    # Creating an instance of the gradient boasting classifier and fitting it with the data to
    #select the features with the most importance 
    clf = GradientBoostingClassifier(random_state=0)
    gbc = clf.fit(train, Y)
    feat_imps = pd.Series(gbc.feature_importances_, index=train.columns)
    # Sorting the features by feature importance
    sorted_feat_imp = feat_imps.sort_values(ascending  = False)
    # Selecting the top 7 features
    top_features = sorted_feat_imp.iloc[:7]
    
    # Creating new train and test dataframe with only these top seven features
    train_set = train[top_features.index]
    test_set = test[top_features.index]
    
    # Slitting the train set into traing and testing data for training the dataset
    X_train, X_test, y_train, y_test = train_test_split(train_set, Y, random_state=0)
    
    # Importing packages necessary to evaluate the model
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    
    # Specifying the list of hyperparameters to fit the model with in order to find the best combination 
    grid_values = {'n_estimators':[2,3], 'learning_rate':[0.001,0.01,0.1,1], 'max_depth':[2,3,4]}
    clf1 = GridSearchCV(GradientBoostingClassifier(),param_grid = grid_values,scoring = 'roc_auc')
    # Fitting the model on the datas set
    clf1.fit(X_train, y_train)
    y_dec_func = clf1.decision_function(X_test)
    
    # Getting the best combination of parameters
    best_values = clf1.best_params_
    # Creating the model with these parameters 
    final_clf = GradientBoostingClassifier(n_estimators = best_values['n_estimators'], 
                                       learning_rate=best_values['learning_rate'], max_depth=best_values['max_depth'])
    # Fitting the model with the data
    final_clf.fit(train_set, Y)
    
    #Using the model to predict the values of the test data
    y_final = final_clf.predict(test_set)
    
    answer = pd.Series(y_final, index=test_set.index.values)
    
    return answer # Your answer here